In [ ]:
!pip install tensorflow

In [ ]:
#Importing Libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [ ]:
# Dataset Paths
data_folder = "/content/drive/MyDrive/Dataset/Signals_GPR"
ctu_df_path = "/content/drive/MyDrive/Dataset/ctu_df.csv"

In [ ]:
# Parameters
sequence_length = 1500
feature_dim = 2

In [ ]:
# Load annotation file
ctu_df = pd.read_csv(ctu_df_path, dtype={'ID': str})  # Taking ID as a string
valid_ids = set(ctu_df['ID'].values)

In [ ]:
X_list = []
y_list = []

In [ ]:
# Processing each CSV file(552 files)
for file in sorted(os.listdir(data_folder)):
    if file.endswith(".csv"):
        file_id = file[:-8]  # Removing ".csv" to get ID
        if file_id not in valid_ids:
            continue  # Skip files not in ctu_df

        # Read CSV (only FHR, UC columns)
        df = pd.read_csv(os.path.join(data_folder, file), usecols=['FHR_GPR', 'UC_GPR'])
        df.dropna(inplace=True)

        # Ensure enough data points
        if len(df) < sequence_length:
            print(f"Skipping {file}, not enough data.")
            continue

        # Select the first `sequence_length` rows
        sequence = df.iloc[:sequence_length].values
        X_list.append(sequence)

        # Assign label from ctu_df
        y_list.append(ctu_df.loc[ctu_df['ID'] == file_id, 'target'].values[0])


In [ ]:
# Convert lists to NumPy arrays
X = np.array(X_list, dtype=np.float32)
y = np.array(y_list, dtype=np.int8)

print(f"Final input shape: {X.shape}")  # Should be (552, sequence_length, 2)
print(f"Final output shape: {y.shape}")  # Should be (552,)

# Save for training
np.save("X.npy", X)
np.save("y.npy", y)

Final input shape: (275, 1500, 2)
Final output shape: (275,)


In [ ]:
# Flatten X for SMOTE (Convert 3D -> 2D)
n_samples, sequence_length, num_features = X.shape
X_flat = X.reshape(n_samples, sequence_length * num_features)

In [ ]:
# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_flat, y)

In [ ]:
# Reshape X back to 3D (time series format)
X_resampled = X_resampled.reshape(-1, sequence_length, num_features)

In [ ]:
# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [ ]:
# LSTM Model
def build_lstm_model(sequence_length, num_features):
    model = keras.Sequential([
        layers.Input(shape=(sequence_length, num_features)),
        layers.LSTM(64, return_sequences=True),
        layers.LSTM(32),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")  # Binary classification
    ])
    return model

In [ ]:
# Compiling the model
model = build_lstm_model(sequence_length, num_features)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# Train Model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.7623 - loss: 0.4361 - val_accuracy: 0.6667 - val_loss: 0.6363
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.8448 - loss: 0.3345 - val_accuracy: 0.6176 - val_loss: 0.7524
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.7899 - loss: 0.4546 - val_accuracy: 0.5784 - val_loss: 0.7302
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.7722 - loss: 0.4785 - val_accuracy: 0.6569 - val_loss: 0.5939
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7951 - loss: 0.4235 - val_accuracy: 0.6765 - val_loss: 0.6174
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8289 - loss: 0.3629 - val_accuracy: 0.6765 - val_loss: 0.6010
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.8894 - loss: 0.3001 - val_accuracy: 0.6863 - val_loss: 0.6403
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8512 - loss: 0.3393 - val_accuracy: 0.6961 - val_loss:

In [ ]:
# Evaluating the model (Calculating the model Loss and Accuracy)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step - accuracy: 0.6588 - loss: 0.6084
Test Accuracy: 0.6471


In [ ]:
# Evaluate Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary (0 or 1)

# Print classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Pathological", "Normal"]))


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 379ms/step

Classification Report:
               precision    recall  f1-score   support

Pathological       0.62      0.76      0.68        51
      Normal       0.69      0.53      0.60        51

    accuracy                           0.65       102
   macro avg       0.66      0.65      0.64       102
weighted avg       0.66      0.65      0.64       102

